### This NoteBook Information
- This Notebook contains the code & steps involved in generating embeddings from array & saving them in npy format for further usuaga in the project

#### Libraries

In [1]:
import numpy as np
import os
import torch # torch is used to deal with nueral network models using gpu
from PIL import Image
from transformers import CLIPProcessor, CLIPModel # this library is used to retrive the CLIP model and processor from hugging face platform
from tqdm import tqdm # this library and function helps to see the progress of the task
import requests # this library is used to check the protocols and proximity issues and connections

C:\Users\vishn\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### GPU Checklist

In [2]:
print(torch.__version__) # mandatory check before fetching the model
print("CUDA available:", torch.cuda.is_available()) # this step check whether the system has cuda enable gpu or not

2.5.1+cu121
CUDA available: True


In [3]:
print(torch.cuda.get_device_name(0)) # this code give the exact nvidia gpu series 

NVIDIA GeForce GTX 1650


In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # selecting the gpu if there is cuda enables gpu on the system
BATCH_SIZE = 1

In [4]:
DEVICE

'cuda'

#### Fixed Variables

In [6]:
TRAIN_IMAGES_PATH = "./Data/Images_Train/"  # images path
EMBEDDING_OUTPUT_DIR = "embeddings" # embeddings folder
MODEL_NAME = "openai/clip-vit-base-patch32"  # CLIP Model path

In [8]:
os.makedirs(EMBEDDING_OUTPUT_DIR,exist_ok=True) # creating the embeddings folder

In [10]:
print(requests.get("https://huggingface.co").status_code) # checking the connection with hugging face 

200


#### Loading Model

In [7]:
model = CLIPModel.from_pretrained(MODEL_NAME, use_safetensors=True).to(DEVICE) # loading model
processor = CLIPProcessor.from_pretrained(MODEL_NAME,use_fast=True) # loading its processor -> decoder

In [15]:
image_files = sorted([
    f for f in os.listdir(TRAIN_IMAGES_PATH)
    if f.lower().endswith((".png", ".jpg", ".jpeg"))
])

embeddings = []
filenames = []

print(f"\n🚀 Generating CLIP embeddings on {DEVICE} for {len(image_files)} images...\n")


🚀 Generating CLIP embeddings on cuda for 60000 images...



#### Embeddings Generation

In [17]:
for i in tqdm(range(0, len(image_files), BATCH_SIZE)):
    batch_files = image_files[i:i + BATCH_SIZE]  # to avoid the gpu usuage limit we are making it in batches
    batch_images = [] 

    for fname in batch_files:
        try:
            img_path = os.path.join(TRAIN_IMAGES_PATH, fname)
            image = Image.open(img_path)
            batch_images.append(image)
        except Exception as e:
            print(f"❌ Skipping {fname}: {e}")
            continue

    inputs = processor(images=batch_images, return_tensors="pt").to(DEVICE) 

    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
        image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)

    for j, fname in enumerate(batch_files):
        embeddings.append(image_features[j].cpu().numpy()) # storing the embeddings and file names
        filenames.append(fname)

# ====== SAVE ======
np.save(os.path.join(EMBEDDING_OUTPUT_DIR, "image_embeddings.npy"), np.array(embeddings))
np.save(os.path.join(EMBEDDING_OUTPUT_DIR, "image_filenames.npy"), np.array(filenames))

print(f"\n✅ Saved {len(embeddings)} embeddings to '{EMBEDDING_OUTPUT_DIR}/'")

100%|██████████████████████████████████████████████████████████████████████████| 60000/60000 [1:12:34<00:00, 13.78it/s]



✅ Saved 60000 embeddings to 'embeddings/'
